<a href="https://colab.research.google.com/github/SirSirocco/DataScience_2025_1/blob/main/experiment_plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEPENDÊNCIAS GERAIS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings


from google.colab import drive
from itertools import product
from sklearn.base import BaseEstimator, clone
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler

# FUNÇÕES

## Utilitário

In [ ]:
def dict_combine(dictionary: dict) -> list:
    """
    Gera todas as combinações possíveis entre os valores de um dicionário de listas,
    retornando uma lista de dicionários com cada combinação única.

    :param dictionary: Dicionário em que cada chave está associada a uma lista de valores possíveis.
                       Os valores devem ser iteráveis (como listas ou tuplas) e de mesmo comprimento.

    :return: Lista contendo dicionários. Cada dicionário representa uma combinação possível
             entre os valores das listas fornecidas no dicionário original. As chaves são mantidas,
             e os valores correspondem a uma das combinações do produto cartesiano.
    """
    # Passo 1: Obtemos as chaves do dicionário (ex: ["key1", "key2"])
    keys = list(dictionary.keys())

    # Passo 2: Obtemos os valores associados a cada chave (listas de possibilidades)
    # Exemplo: [["a", "b", "c"], ["d", "e", "f"]]
    values = list(dictionary.values())

    # Passo 3: Calculamos o produto cartesiano dessas listas de valores
    # Isso gera todas as combinações possíveis, como ("a", "d"), ("a", "e"), ..., ("c", "f")
    combinations = product(*values)  # Asterisco faz o desempacotamento dos elementos de values

    # Passo 4: Para cada combinação, associamos os valores às suas respectivas chaves
    # Isso é feito com a função zip, e transformamos o resultado em um dicionário
    # Resultado final: uma lista de dicionários, cada um representando uma combinação possível
    return [dict(zip(keys, combo)) for combo in combinations]

def dict_flatten(dictionary: dict, fields_to_flatten: list[str]) -> dict:
    """
    Desestrutura os campos selecionados de um dicionário, retornando apenas os atributos desejados.

    :param dictionary: Dicionário original que contém os campos a serem desestruturados.
    :param fields_to_flatten: Lista de nomes dos campos que devem ser desestruturados (flatten).

    :return: Novo dicionário contendo apenas os atributos dos campos selecionados, combinados em um nível.
    """
    dict_flat = dict()

    for field in fields_to_flatten:
        value = dictionary.get(field)
        if isinstance(value, dict):
            dict_flat.update(value)

    return dict_flat

def dict_to_flat_df(dictionary: dict, fields_to_flatten: list[str], key_name: str = "key") -> pd.DataFrame:
    """
    Converte um dicionário com estrutura aninhada em um DataFrame tabular, desestruturando
    apenas os campos especificados.

    :param dictionary: Dicionário em que cada chave representa um identificador único e cada valor é um dicionário com possíveis campos aninhados.
    :param fields_to_flatten: Lista de nomes de campos a serem desestruturados. Cada um deve
                               corresponder a uma chave cujo valor é um dicionário.
    :param key_name: Nome da coluna que armazenará os identificadores do dicionário original.
                     Padrão é "key".

    :return: DataFrame com uma coluna para os identificadores e colunas adicionais contendo os
             atributos resultantes da desestruturação dos campos selecionados.
    """
    rows = list()

    for identifier, nested_dict in dictionary.items():
        dict_flat = dict_flatten(nested_dict, fields_to_flatten)
        dict_flat[key_name] = identifier
        rows.append(dict_flat)

    df = pd.DataFrame(rows)
    df = df[[key_name] + [col for col in df.columns if col != key_name]]  # Coloca a chave primeiro
    return df

def key_build(key_parts: list[str], separator: str = "_") -> str:
    return separator.join(key_parts)

def key_build_from_params(params: dict[str]):
    parts = [key_build([str(k).upper(), str(v).lower()]) for k, v in params.items()]
    return key_build(parts)

## Exibição

In [ ]:
def df_show_head(df: pd.DataFrame, n: int = 5) -> None:
    display(df.head(n))
    print(f"Shape: {df.shape}")

## Validação

In [ ]:
def is_valid_col_selection(col_selection: dict) -> bool:
    return col_selection is not None

def is_valid_series(se: pd.Series) -> bool:
    return se is not None and (not se.empty) and se.notnull().all() and pd.api.types.is_numeric_dtype(se)

## Pré-Processamento

In [ ]:
def subs_outliers(df: pd.DataFrame, columns: list[str], view: bool = False) -> pd.DataFrame:
    """
    Substitui outliers pela média. Se view for True, exibe
    dois gráficos para cada coluna: antes e depois da mudança.
    """

    # Guarda resultado anterior
    df_before = df.copy()

    for column in columns:
        # Obtém quartis 1 e 3
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)

        # Obtém IQR (Intervalo Interquartílico)
        iqr = q3 - q1

        """
        Outlier se em (-inf, Q1 - 1.5 * IQR) ou (Q3 + 1.5 * IQR, +inf).
        Subsititui outliers por NA ('not available') segundo método IQR (InterQuartile Range).
        """
        df.loc[(df[column] < q1 - 1.5 * iqr) | (df[column] > q3 + 1.5 * iqr), column] = pd.NA

        # Substitui NAs pela média
        df[column] = df[column].fillna(df[column].mean())

    if view:
        for column in columns:
            # Exibe resultado inicial
            plt.boxplot(df_before[column])
            plt.title(f"ANTES: {column}")
            plt.show()

            # Exibe novo resultado
            plt.boxplot(df[column])
            plt.title(f"DEPOIS: {column}")
            plt.show()

    return df

## Validação Cruzada

In [ ]:
def cross_val(model: BaseEstimator,
              X: pd.DataFrame,
              y: pd.Series,
              folds: int = 5,
              metric: str = "r2",
              shuffle: bool = True,
              random_state: int = None) -> tuple[list, list]:
    """
    Realiza validação cruzada utilizando K-Fold para treinar e avaliar um modelo,
    com base em uma métrica de desempenho fornecida.

    O modelo é clonado para cada divisão dos dados, garantindo independência entre os folds.

    :param model: Estimador compatível com o Scikit-learn (deve implementar os métodos `fit` e `predict`).
    :param X: Conjunto de dados de entrada (features).
    :param y: Vetor alvo (target) correspondente às amostras em `X`.
    :param folds: Número de partições (folds) na validação cruzada. O padrão é 5.
    :param metric: Métrica de avaliação utilizada em cada fold.
                   Pode ser uma das opções: `"r2"`, `"mse"`, `"mae"`.
                   Se inválida, `"r2"` será usada como padrão.
    :param shuffle: Indica se os dados devem ser embaralhados antes da divisão em folds.
    :param random_state: Semente de aleatoriedade para reprodutibilidade. `None` implica aleatoriedade completa.

    :return: Tupla contendo:
             - lista de modelos treinados (um por fold)
             - lista de métricas correspondentes a cada fold
    """
    DICT_METRICS = {
        "r2":   r2_score,
        "mse":  mean_squared_error,
        "mae":  mean_absolute_error
    }
    models = list() # Salva modelos por fold
    metrics = list() # Salva resultados por fold
    metric_to_use = DICT_METRICS.get(metric, r2_score) # Define métrica a ser usada (padrão é coeficiente de determinação)

    # Define os índices que delimitam as partições
    kfolds = KFold(n_splits=folds, shuffle=shuffle, random_state=random_state)

    for train_index, test_index in kfolds.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        copy = clone(model) # Evita sobrescrição do modelo
        copy.fit(X_train, y_train)

        models.append(copy)
        metrics.append(metric_to_use(y_test, copy.predict(X_test))) # Calcula métrica

    return models, metrics

## Casos

In [ ]:
# --- CONSTANTES SIMBÓLICAS ---
"""
Incluímos '_' ao início para evitar conflitos de nomes com o restante do código.
"""
# - Pré-processamento -
_COLUMN_REMOVAL = "col_rem"
_COLUMNS = "columns"
_NAME = "name"
_FEATURE_ENGINEERING = "feat_eng"
_OUTLIERS = "outliers"
_NORMALIZATION = "normalization"
_PCA = "pca"

PREFIXES = {
    _COLUMN_REMOVAL: "FS",
    _FEATURE_ENGINEERING: "FE",
    _OUTLIERS: "OUT",
    _NORMALIZATION: "NORM",
    _PCA: "PCA"
}

_ABSENT = "absent"
_TRUE = "true"
_FALSE = "false"

RADIXES = {
    _ABSENT: "na",
    _TRUE: "1",
    _FALSE: "0"
}

_X = "X"
_y = "y"
_PARAMS = "params"

# - Métricas -
_MAE = "mae"
_MSE = "mse"
_R2 = "r2"
DICT_METRICS = {
        _MAE: mean_absolute_error,
        _MSE: mean_squared_error,
        _R2:  r2_score,
}
_AVERAGE = "avg"
_BEST_MODEL = "best_model"
_CROSS_VAL = "cv"
_TEST = "test"

# FUNÇÕES
def apply_column_removal(df, X, y, col_selection, prefix, radixes):
    key = prefix

    if is_valid_col_selection(col_selection):
        columns = col_selection[_COLUMNS]
        df = df.drop(columns=columns)
        key = key_build([key, col_selection[_NAME].lower()]) # Indica nome da seleção de colunas
    else:
        key = key_build([key, radixes[_ABSENT]])
        columns = None

    return df, X, y, key, {_COLUMN_REMOVAL: columns}

def apply_feature_engineering(df, X, y, feat_eng, prefix, radixes):
    key = prefix

    if is_valid_series(feat_eng):
        df = pd.concat([df, feat_eng], axis=1)
        key = key_build([key, feat_eng.name.lower()]) # Indica nome da característica
        feat_name = feat_eng.name
    else:
        key = key_build([key, radixes[_ABSENT]])
        feat_name = None

    return df, X, y, key, {_FEATURE_ENGINEERING: feat_name}

def apply_outliers(df, X, y, out, prefix, radixes):
    key = prefix

    if out:
        df = subs_outliers(df, df.columns, view=False)
        key += radixes[_TRUE]
    else:
        key += radixes[_FALSE]

    X = df.drop(columns=[y.name])
    y = df[y.name].squeeze() # Squeeze garante ser uma série

    return df, X, y, key, {_OUTLIERS: out}

apply_normalization_scaler = StandardScaler() # Guarda scaler para reuso
def apply_normalization(df, X, y, norm, prefix, radixes):
    key = prefix

    if norm:
        X = pd.DataFrame(apply_normalization_scaler.fit_transform(X), columns=X.columns, index=X.index)
        key += radixes[_TRUE]
    else:
        key += radixes[_FALSE]

    return df, X, y, key, {_NORMALIZATION: norm}

def apply_pca(df, X, y, pca_n, prefix, radixes):
    key = f"{prefix}{pca_n}"

    # Evita que componentes superem total de colunas, o que geraria redundância
    pca_n = min(len(X.columns), pca_n)

    pca_obj = PCA(n_components=pca_n)
    pca_arr = pca_obj.fit_transform(X)
    X_pca = pd.DataFrame(
        pca_arr,
        index=X.index,
        columns=[f"pc{i + 1}" for i in range(pca_n)]
    )

    return df, X_pca, y, key, {_PCA: pca_n}

def get_preprocessing_cases(df: pd.DataFrame,
                            target: str,
                            random_state: int,
                            col_selections: list[dict],
                            features: list[pd.Series],
                            outliers: list[bool],
                            normalization: list[bool],
                            pcas: list[int]) -> dict:
    """
    Gera múltiplos cenários de pré-processamento combinando, seleção e engenharia de atributos,
    tratamento de outliers, normalização e redução de dimensionalidade (PCA).

    Cada cenário gera um conjunto `X`, `y` e um dicionário com parâmetros das transformações aplicadas.
    As chaves do dicionário final descrevem o pipeline aplicado via prefixos e sufixos codificados.

    :param df: DataFrame original contendo os dados brutos.
    :param target: Nome da coluna alvo (variável dependente) no DataFrame.
    :param random_state: Semente para controle de aleatoriedade (reprodutibilidade).
    :param col_selection: Lista de dicionários com atributos "name" (representação do cenário) e "columns" (lista de colunas a serem removidas).
    :param features: Lista de séries representando atributos derivados para engenharia de características.
                     Pode conter séries válidas ou `None` para ausência de feature engineering.
    :param outliers: Lista indicando se o tratamento de outliers deve ser aplicado.
    :param normalization: Lista indicando se a normalização (z-score) deve ser aplicada.
    :param pcas: Lista com números de componentes principais a serem testados via PCA.

    :return: Dicionário cujas chaves são strings descritivas do pipeline (ex: 'FEna_OUT1_NORM0_PCA2'), e
             os valores são dicionários contendo:
             - "X": DataFrame com os atributos processados.
             - "y": Série da variável alvo.
             - "params": Parâmetros que descrevem as transformações aplicadas no cenário.
    """
    dict_cases = dict() # Armazena os casos de experimentação
    df_original = df.copy() # Salva estado original do DataFrame

    np.random.seed(random_state) # Fixa semente aleatória

    for select, feat, out, norm, pca in product(col_selections, features, outliers, normalization, pcas):
        # Etapas de pré-processamento
        steps = [
            (apply_column_removal, select, PREFIXES[_COLUMN_REMOVAL]),
            (apply_feature_engineering, feat, PREFIXES[_FEATURE_ENGINEERING]),
            (apply_outliers, out, PREFIXES[_OUTLIERS]),
            (apply_normalization, norm, PREFIXES[_NORMALIZATION]),
            (apply_pca, pca, PREFIXES[_PCA]),
        ]

        key_parts = list()          # Armazena partes da chave que descreve o caso
        dict_params = dict()        # Armazena os parâmetros que compõem o caso
        df = df_original.copy()     # Copia DataFrame original para evitar sobrescrição
        X = df.drop(columns=[target])
        y = df[target]

        # Executa etapas
        for function, param, prefix in steps:
            df, X, y, key, update = function(df, X, y, param, prefix, RADIXES)
            key_parts.append(key)
            dict_params.update(update)

        key = key_build(key_parts)
        dict_cases[key] = {_X: X, _y: y, _PARAMS: dict_params}

    return dict_cases

def get_best_model(metric, models, scores):
    DICT_CRITERIA = {
        _MAE: np.argmin,
        _MSE: np.argmin,
        _R2: np.argmax
    }
    best_index = DICT_CRITERIA[metric](scores)
    best_index = best_index[0] if isinstance(best_index, np.ndarray) else best_index
    return models[best_index]

def get_model_cases(model: BaseEstimator,
                    pre_processing_cases: dict,
                    params: dict,
                    folds: list[int],
                    metrics: list[str],
                    test_size: float,
                    random_state: int) -> dict:
    dict_cases = dict()
    model_name = model.__class__.__name__

    # Percorre cenários de pré-processamento
    for base_key, pre_proc_params in pre_processing_cases.items():
        # Inicio da nova chave formado pela chave do pré-processamento e nome do modelo
        base_key = key_build([base_key, model_name])

        # Gera conjuntos de treino e de teste
        X = pre_proc_params[_X]
        y = pre_proc_params[_y]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

        # Gera cenários com base no número de folds e nos parâmetros do modelo
        for model_params, fold_num in product(dict_combine(params), folds):
            dict_best_models = dict() # Armazena melhores modelos por métrica
            dict_scores_cross_val = dict() # Armazena pontuações médias da V.C. por métrica
            dict_scores_test = dict() # Armazena pontuações do teste por métrica

            model_case = clone(model)
            model_case.set_params(**model_params)

            for metric in metrics:
                models_cross_val, scores_cross_val = cross_val(model_case,
                                    X_train,
                                    y_train,
                                    folds=fold_num,
                                    metric=metric,
                                    shuffle=True,
                                    random_state=random_state)

                # Computa métrica média
                key_cross_val_avg = key_build([metric, _CROSS_VAL, _AVERAGE])
                dict_scores_cross_val[key_cross_val_avg] = np.mean(scores_cross_val)

                # Escolhe melhor modelo
                key_cross_val_best_model = key_build([metric, _CROSS_VAL, _BEST_MODEL])
                best_model = get_best_model(metric, models_cross_val, scores_cross_val)
                dict_best_models[key_cross_val_best_model] = best_model

                # Computa métricas do melhor modelo
                y_pred = best_model.predict(X_test)
                for internal_metric in metrics:
                    key_test = key_build([key_cross_val_best_model, _TEST, internal_metric])
                    dict_scores_test[key_test] = DICT_METRICS[internal_metric](y_test, y_pred)

            key_model_params = key_build_from_params(model_params)
            key_new = key_build([base_key, key_model_params])

            dict_cases[key_new] = {
                _X: X,
                _y: y,
                _PARAMS: {**model_params},
                _BEST_MODEL: dict_best_models,
                _CROSS_VAL: dict_scores_cross_val,
                _TEST: dict_scores_test
            }

    return dict_cases

# SETUP DO AMBIENTE

In [ ]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# OBTENÇÃO DO DATASET

In [ ]:
PATH = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets/cooked/_all/all_merged.csv"
df = pd.read_csv(PATH)
df_show_head(df)

,_ano,_estado,_mes,car_c02_emitido,cli_pressao_atm_med,cli_temp_ar_med,cli_temp_orvalho_med,cli_umid_rel_med,cli_umid_rel_min_max,cli_umid_rel_min_med,cli_umid_rel_min_min,cli_veloc_vento_max,cli_veloc_vento_med,que_area_queimada,que_focos_qtd
0,2008,AC,7,2.627698e+07,986.843612,28.142731,18.914978,59.555066,95.0,54.432558,29.0,5.1,2.152915,4957.0,165.0
1,2008,AC,9,2.627698e+07,991.705941,24.446194,19.467987,75.811881,97.0,72.496700,25.0,1.0,0.210504,46073.0,2947.0
2,2008,AC,10,2.627698e+07,990.328360,25.229298,21.617473,81.870968,96.0,78.720430,29.0,1.0,0.204959,30355.0,856.0
3,2008,AC,11,2.627698e+07,988.610987,25.195410,22.624478,86.905424,96.0,84.048679,42.0,1.0,0.186970,2082.0,63.0
4,2008,AC,12,2.627698e+07,988.692608,24.898790,22.727554,88.529570,96.0,86.116935,53.0,1.0,0.179442,127.0,4.0


Shape: (1025, 15)


# CODIFICAÇÃO

In [ ]:
# Codificação OneHot simplificada
TO_ENCODE = ["_estado"]
df_encoded = pd.get_dummies(df, columns=TO_ENCODE, dtype="Int32")

df_show_head(df_encoded)

,_ano,_mes,car_c02_emitido,cli_pressao_atm_med,cli_temp_ar_med,cli_temp_orvalho_med,cli_umid_rel_med,cli_umid_rel_min_max,cli_umid_rel_min_med,cli_umid_rel_min_min,...,cli_veloc_vento_med,que_area_queimada,que_focos_qtd,_estado_AC,_estado_AM,_estado_AP,_estado_PA,_estado_RO,_estado_RR,_estado_TO
0,2008,7,2.627698e+07,986.843612,28.142731,18.914978,59.555066,95.0,54.432558,29.0,...,2.152915,4957.0,165.0,1,0,0,0,0,0,0
1,2008,9,2.627698e+07,991.705941,24.446194,19.467987,75.811881,97.0,72.496700,25.0,...,0.210504,46073.0,2947.0,1,0,0,0,0,0,0
2,2008,10,2.627698e+07,990.328360,25.229298,21.617473,81.870968,96.0,78.720430,29.0,...,0.204959,30355.0,856.0,1,0,0,0,0,0,0
3,2008,11,2.627698e+07,988.610987,25.195410,22.624478,86.905424,96.0,84.048679,42.0,...,0.186970,2082.0,63.0,1,0,0,0,0,0,0
4,2008,12,2.627698e+07,988.692608,24.898790,22.727554,88.529570,96.0,86.116935,53.0,...,0.179442,127.0,4.0,1,0,0,0,0,0,0


Shape: (1025, 21)


# ENGENHARIA DE CARACTERÍSTICAS

In [ ]:
"""
Para capturar a intensidade dos focos de incêndio, dividimos a área total queimada
pela quantidade de focos. Talvez essa característica derivada revele nuances aos modelos
que não seriam perceptíveis pela consideração individualizada de ambos os atributos.
"""
se_intensity = df_encoded["que_area_queimada"] / (df_encoded["que_focos_qtd"])
se_intensity.name = "que_focos_intensidade"
display(se_intensity)

,que_focos_intensidade
0,30.042424
1,15.633865
2,35.461449
3,33.047619
4,31.750000
...,...
1020,281.112392
1021,154.337561
1022,101.463881
1023,396.212500


# SELEÇÃO DE COLUNAS

In [ ]:
scn_not_corr = {
    "name": "not_corr",
    "columns": [
        "cli_temp_orvalho_med",
        "cli_umid_rel_med",
        "cli_umid_rel_min_min",
        "cli_veloc_vento_max"
    ]
}

scn_stateless = {
    "name": "stateless",
    "columns": [col for col in df_encoded.columns if col.startswith("_estado")]
}

# SETUP

In [ ]:
# Constantes simbólicas
PRE =   "pre"     # Pré-processamento
RF =    "randfor" # Random Forest
XGB =   "xgboost" # XGBoost
MLR =   "mlinreg" # Regressão Linear Múltipla
ALL =   "all"     # Todos os modelos considerados simultaneamente

# Dicionários para armazenar os resultados
dict_cases = dict()
dict_df_cases = dict()

# PARÂMETROS GERAIS
RANDOM_SEED = 42
TEST_SIZE = 0.30
TARGET = "que_area_queimada"

# CASOS

## CASOS DE PRÉ-PROCESSAMENTO

In [ ]:
# Parâmetros auxiliares (mude se desejar)
SCENARIOS = [None, scn_not_corr , scn_stateless]
FEATURE_ENGINEERING = [None, se_intensity]
OUTLIERS = [False]
NORMALIZATION = [False, True]
PCA_LIST = [2]

# Obtém casos de pré-processamento
dict_cases[PRE] = get_preprocessing_cases(df_encoded,
                        TARGET,
                        RANDOM_SEED,
                        SCENARIOS,
                        FEATURE_ENGINEERING,
                        OUTLIERS,
                        NORMALIZATION,
                        PCA_LIST
                        )
dict_df_cases[PRE] = pd.DataFrame(data=dict_cases[PRE]).T

df_show_head(dict_df_cases[PRE])

,X,y,params
COLRM_na_FE_na_OUT0_NORM0_PCA2,"[[-81243294.26560223, -361.8358884636149], [-8...",0 4957.0 1 46073.0 2 303...,"{'col_rem': None, 'feat_eng': None, 'outliers'..."
COLRM_na_FE_na_OUT0_NORM1_PCA2,"[[-2.2260733347087305, -0.41725887679653006], ...",0 4957.0 1 46073.0 2 303...,"{'col_rem': None, 'feat_eng': None, 'outliers'..."
COLRM_na_FE_que_focos_intensidade_OUT0_NORM0_PCA2,"[[-81243294.27942637, -5938.757289311726], [-8...",0 4957.0 1 46073.0 2 303...,"{'col_rem': None, 'feat_eng': 'que_focos_inten..."
COLRM_na_FE_que_focos_intensidade_OUT0_NORM1_PCA2,"[[-2.2125548335693908, -0.5113594837091535], [...",0 4957.0 1 46073.0 2 303...,"{'col_rem': None, 'feat_eng': 'que_focos_inten..."
COLRM_not_corr_FE_na_OUT0_NORM0_PCA2,"[[-81243294.26560195, -361.9206619281286], [-8...",0 4957.0 1 46073.0 2 303...,"{'col_rem': ['cli_temp_orvalho_med', 'cli_umid..."


Shape: (12, 3)


## CASOS DE KNN

In [ ]:
# Parâmetros auxiliares (mude se desejar)
PARAMS = {
    "n_estimators": [100],
    "criterion": ["squared_error"],
    "max_depth": [2, 10],
    "random_state": [RANDOM_SEED]
}
model = RandomForestRegressor()

# Obtém casos de KNN
dict_cases[RF] = get_model_cases(model,
                                 dict_cases[PRE],
                                 PARAMS,
                                 [3, 5, 10],
                                 ["r2", "mse", "mae"],
                                 TEST_SIZE,
                                 RANDOM_SEED
                               )
dict_df_cases[RF] = pd.DataFrame(data=dict_cases[RF]).T

df_show_head(dict_df_cases[RF])

# GERAÇÃO DA PLANILHA

In [ ]:
FLATTEN = ["params", "avg_scores_cross_val", "scores_test"]
KEY_NAME = "case"
dict_df_final = dict()

for key, value in dict_cases.items():
    dict_df_final[key] = dict_to_flat_df(value, FLATTEN, KEY_NAME)
    # df_show_head(dict_df_final[key])

cases_to_cmp = [dict_df_final[key] for key in [RF]]
dict_df_final[ALL] = pd.concat(cases_to_cmp, axis=0).reset_index(drop=True)
dict_df_final[ALL].dropna(axis=1, inplace=True)

df_show_head(dict_df_final[ALL])

In [ ]:
# Caminho do arquivo
PATH_OUT = "experiment_plan.xlsx"

# Salva em múltiplas abas
with pd.ExcelWriter(PATH_OUT, engine="openpyxl") as writer:
    for sheet_name, df in dict_df_final.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)